In [18]:
!pip install ..
!pip install rouge-score

Processing /home/hieunm/TextMining/multidocsum
  Created wheel for multidocsum: filename=multidocsum-0.0.1-py3-none-any.whl size=8431 sha256=eebe0c130db00e2a2e180fbeb69aae901bf7a7e1974964cf771a731dcbdc9b66
  Stored in directory: /tmp/pip-ephem-wheel-cache-txjzhb8d/wheels/c6/cd/5a/9ec9ef8017e7bde05e4cfc0305ff43fb2204ade629ffb3d836
Successfully built multidocsum
  Attempting uninstall: multidocsum
    Found existing installation: multidocsum 0.0.1
    Uninstalling multidocsum-0.0.1:
      Successfully uninstalled multidocsum-0.0.1


In [20]:
from src.models.extracting import Extractor
from src.data.data_io import DataIO
import numpy as np
from glob import glob

In [12]:
cluster_name = []
for path in glob('../data/interim/extract_content/*'):
    cluster_name.append(path.split('/')[-1])

In [21]:
my_sum = {}
gold_sum = {}
for name in cluster_name[:3]:
    params = {
        'data_path': f'../data/interim/extract_content/{name}',
        'stop_words_path': '../data/external/stopwords.txt', 
        'punc_path': '../data/external/punctuation.txt',
        'compression_rate': 0.05,
        'remove_redundacy': True,
        'wc': 1.0, 
        'wp': 1.0, 
        'wf': 1.0, 
        'n_centroid': 10, 
        'C_max': 10, 
        'ngram_range': (1, 1), 
        'max_features': 100,
        'func': np.dot
    }
    summary = Extractor.extract(**params)
    os.makedirs(f'../../predict/{name}', exist_ok=True)
    with open(f'../../predict/{name}/summary.txt', 'w') as f:
        f.write('\n'.join(summary))
        my_sum[name] = '\n'.join(summary)
        gold_sum[name] = DataIO.load(f'../data/raw/ViMs/summary/{name}')

In [30]:
from rouge_score import rouge_scorer

In [38]:
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rouge3', 'rouge4'])

In [58]:
mean = {}
for cluster in my_sum:
    mean[cluster] = {}
    precision = []
    precision.append(scorer.score(gold_sum[cluster]['0.gold.txt'], my_sum[cluster])['rouge1'].precision)
    mean[cluster]['precision'] = precision[0]    
    recall = []
    recall.append(scorer.score(gold_sum[cluster]['0.gold.txt'], my_sum[cluster])['rouge1'].recall)
    mean[cluster]['recall'] = recall[0]

In [59]:
mean

{'Cluster_060': {'precision': 0.8916967509025271,
  'recall': 0.5995145631067961},
 'Cluster_108': {'precision': 0.5578947368421052,
  'recall': 0.8091603053435115},
 'Cluster_271': {'precision': 0.4578313253012048,
  'recall': 0.9606741573033708}}